<a href="https://colab.research.google.com/github/Snapchik/chat_analysis/blob/main/Chats_copmparisson.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [28]:
import pandas as pd
import re
import numpy as np
import matplotlib.pyplot as plt
import string
import nltk

from nltk.corpus import stopwords          # module for stop words that come with NLTK
from nltk.stem import PorterStemmer        # module for stemming

In [29]:
# download the stopwords from NLTK
nltk.download('stopwords')

nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [30]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [31]:
hammas_chat = pd.read_json('/content/drive/My Drive/Colab Notebooks/result.json')

hammas_chat['hashtags'] = [[] for _ in range(len(hammas_chat))]
hammas_chat['date'] = [[] for _ in range(len(hammas_chat))]
hammas_chat['clear_message'] = [[] for _ in range(len(hammas_chat))]

def extract_hashtags(messages):
    try:
        return re.findall(r'#(\w+)', str(messages))[0]
    except IndexError as err:
        return np.nan


hammas_chat['hashtags'] = hammas_chat['messages'].apply(extract_hashtags)

message_count = len(hammas_chat['messages'])

for index in range(message_count):
  hammas_chat['date'][index] = pd.to_datetime(hammas_chat.iloc[index, 3]['date'])

for index in range(message_count):
  hammas_chat['clear_message'][index] = hammas_chat.iloc[index, 3]['text']



hammas_chat = hammas_chat.drop(columns=['name', 'type', 'id'])
hammas_chat = hammas_chat.replace('', np.nan)
hammas_chat = hammas_chat.apply(lambda y: np.nan if len(y)==0 else y)
hammas_chat = hammas_chat.dropna(subset = ['clear_message'])

Streaming output truncated to the last 5000 lines.
<ipython-input-31-05253a1a3b2b>:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hammas_chat['clear_message'][index] = hammas_chat.iloc[index, 3]['text']
<ipython-input-31-05253a1a3b2b>:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hammas_chat['clear_message'][index] = hammas_chat.iloc[index, 3]['text']
<ipython-input-31-05253a1a3b2b>:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-cop

In [32]:
# WHY THIS CODE WORKS AND NOT THE ONE ON TOP
for j in range(len(hammas_chat)):
  lst = []
  for i in hammas_chat.iloc[j, 3]:
    try:
      lst.extend(i.values())
    except AttributeError as err:
      lst.extend(i)
  lst = ''.join(lst)
  #getting rid of typetext/n
  typetext = '\w+\n'
  re.sub(typetext,'', lst)
  hammas_chat.iloc[j, 3] = lst

In [66]:
#clearing out some common misspellings and hyperlink
hammas_chat['clear_message'] = hammas_chat['clear_message'].str.replace('\n', '')
hammas_chat['clear_message'] = hammas_chat['clear_message'].str.replace('text_link', '')
hammas_chat['clear_message'] = hammas_chat['clear_message'].str.replace(r'https?://[^\s\n\r]+', '', regex=True)
hammas_chat['clear_message'] = hammas_chat['clear_message'].str.replace(r'bold\B', '', regex=True)
hammas_chat['clear_message'] = hammas_chat['clear_message'].str.replace(r'#', '',regex=True)
hammas_chat['clear_message'] = hammas_chat['clear_message'].str.replace('hashtag', '')
hammas_chat['clear_message'] = hammas_chat['clear_message'].str.replace('italic', '')
hammas_chat['clear_message'] = hammas_chat['clear_message'].str.replace('_', '')
hammas_chat['clear_message'] = hammas_chat['clear_message'].str.replace('—', '')


In [67]:
#applying tokenizer
hammas_chat['messages_token'] = hammas_chat["clear_message"].apply(nltk.word_tokenize)

In [68]:
stopwords_english = stopwords.words('english')

print('Stop words\n') #\n adds new line
print(stopwords_english)

print('\nPunctuation\n') #here new line added before and after print statement
print(string.punctuation)

Stop words

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so

In [69]:
hammas_chat['tokens_without_stopwords'] = hammas_chat['messages_token'].apply(lambda x:
    [word for word in x if word not in (stopwords_english) and word not in string.punctuation])



In [70]:
# Instantiate stemming class
stemmer = PorterStemmer()

hammas_chat['stemmed_tokens_wt_stopwords_punctuation'] = hammas_chat['tokens_without_stopwords'].apply(lambda x: [stemmer.stem(word) for word in x])

print(hammas_chat['stemmed_tokens_wt_stopwords_punctuation'])

2        [welcom, joint, channel, creat, order, deliv, ...
4        [coloni, occup, forc, fire, tear, ga, youth, b...
5        [young, men, shuafat, shave, head, attempt, co...
6        [yesterday, pa, forc, arrest, ana, qafisha, mi...
7        [the, lionsden, nablu, releas, statement, toda...
                               ...                        
30366    [🚨, humanitarian, aid, airdrop, jordanian, pla...
30367    [🔴, martyr, abu, ali, mustafa, brigad, the, ma...
30368    [🔴, popular, front, liber, palestin, on, step,...
30369    [⚪️, mujahideen, brigad, in, respons, corrupt,...
30370    [🚨, palestinian, ministri, health, the, toll, ...
Name: stemmed_tokens_wt_stopwords_punctuation, Length: 25566, dtype: object


In [75]:
hammas_chat.iloc[7]

messages                                   {'id': 39, 'type': 'message', 'date': '2022-10...
hashtags                                                                              Nablus
date                                                                     2022-10-16 00:23:39
clear_message                              🚨 Nablus | Occupation forces fired upon the re...
messages_token                             [🚨, Nablus, |, Occupation, forces, fired, upon...
tokens_without_stopwords                   [🚨, Nablus, Occupation, forces, fired, upon, r...
stemmed_tokens_wt_stopwords_punctuation    [🚨, nablu, occup, forc, fire, upon, resist, fi...
Name: 10, dtype: object

Next up I need to create a util package like in lab